# Plan
\
Steps:
- (If applicable) Select `event_type` to study (gunfire, mp, one more?) and filter for those records
- Select subset of all `district`s to consider
    - 5 selections + Other? So 6 total?
    - I think there are a few named in the "analyze_oemc" notebook
- Group by `district` (the raw version from OEMC? Using `numeric_district` atm)
- Calculate the rate of non-missing `disp_date` values per group (should be an indicator setup for this already)
- Test the observed rate of non-missing dispatch dates per district for this emergency type against the assumption / null hypothesis that dispatch is reported at equal rates for all districts
    - Per the `power_divergence` docs, if we do not supply an argument for f_exp (expected frequencies), “the categories are assumed to be equally likely.” So we don’t have to calculate that


On districts chosen,
- Trina had recommended looking for districts ['2', '8', '9', '19', '20', '24'] when we looked at [MP districts](https://github.com/HRDAG/US-II-MP/blob/main/individual/MP_CPD/explore/note/districts-plots.ipynb)
- [District map](https://news.wttw.com/sites/default/files/Chicago%20Police%20Districts.pdf) from LB's perplexity query
- Here we've selected
    - 2
    - 7 (district 7 includes Englewood per WTTW map and has 5th highest event count)
    - 8 (district 8 has the 2nd highest event count)
    - 9
    - 19
    - 20
    - 24
- and all other districts are grouped as "Other"

Resources:
- [write-declaration](https://github.com/HRDAG/SF-PDO-RJA/blob/main/write/memos/DA/ACLU/src/write-declaration.py) has chi-square tests I used in RJA declarations

# setup

In [1]:
# dependencies
import yaml
import pandas as pd
from scipy.stats import chisquare, power_divergence

In [2]:
# support methods
def readyaml(fname):
    with open(fname, "r") as f:
        rules = yaml.safe_load(f)
    return rules


def regroup_td(td):
    if td < pd.Timedelta(minutes=5): return 'Under 5 min'
    elif td < pd.Timedelta(minutes=15): return 'Under 15 min'
    elif td < pd.Timedelta(minutes=30): return 'Under 30 min'
    elif td < pd.Timedelta(minutes=60): return 'Under 60 min'
    elif td >= pd.Timedelta(minutes=60): return 'Over 60 min'


def format_longfloat(prop, asperc=False):
    if not asperc: return float(f'{prop:.3f}')
    else: return float(f'{prop*100:.1f}')


def lookup_cramer_cat(msrmt, df):
    if df == 1:
        if msrmt <= 0.1: return 'small'
        elif msrmt <= 0.3: return 'medium'
        else: return 'large'
    elif df == 2:
        if msrmt <= 0.07: return 'small'
        elif msrmt <= 0.21: return 'medium'
        else: return 'large'
    elif df == 3:
        if msrmt <= 0.06: return 'small'
        elif msrmt <= 0.17: return 'medium'
        else: return 'large'
    elif df == 4:
        if msrmt <= 0.05: return 'small'
        elif msrmt <= 0.15: return 'medium'
        else: return 'large'
    elif df == 5:
        if msrmt <= 0.04: return 'small'
        elif msrmt <= 0.13: return 'medium'
        else: return 'large'


def cramers(stat, n, df):
    msrmt = np.sqrt(stat/(n*df))
    cat = lookup_cramer_cat(msrmt=msrmt, df=df)
    return format_longfloat(msrmt), cat


def text_statistic(pcs, sig, n, dof, null_phrase, exp=None):
    rounded_stat = format_longfloat(pcs.statistic)
    rounded_pval = format_longfloat(pcs.pvalue)
    if rounded_pval == 0: rounded_pval = "< 0.0001"
    if exp: newinfo = f"\nThis test results in a p-value of {rounded_pval}, "
    else: newinfo = f"\nThis test results in a p-value of {rounded_pval}, "
    if pcs.pvalue < sig:
        effect_size, effect_cat = cramers(pcs.statistic, n, dof)
        newinfo += f"which is a statistically significant difference and rejects the null hypothesis that {null_phrase}. The test indicates a {effect_cat} effect. "
    else: newinfo += f"which is not statistically significant and fails to reject the null hypothesis that {null_phrase}. "
    if n < 30: newinfo += f"However, the sample size of {n} is small."
    return newinfo


def get_chisquare_overall(df, districts, sig=0.05, ddof=0):
    assert 'dispatch_reported' in df.columns
    sample_size = df.dispatch_reported.sum() # is this right? or should it be df.shape[0]?
    obs_n = [df.loc[df.numeric_district == district, 'dispatch_reported'].sum()/sample_size
             for district in districts]
    dof = len(districts) - 1 - ddof
    #nobs = sum(obs_n) # why use this and not sample_size?
    pcs = power_divergence(f_obs=obs_n, ddof=ddof, lambda_ = "pearson")
    chi_info = f"""This test will compare whether observed dispatch rates are equal across districts. In interpreting the results, a p-value below {sig} will be considered statistically significant. In such cases where the p-value is statistically significant, the effect size will be measured and categorized as small, medium, or large.\n"""
    suppinfo = text_statistic(pcs, sig=sig, n=sample_size, dof=dof,
                             null_phrase=f"the distribution by district of emergency events with a reported dispatch date are equal")
    info = chi_info + suppinfo + "\n\n\n"
    return obs_n, pcs, info


def get_chisquare_event(df, districts, eventtype, sig=0.05, ddof=0):
    assert 'dispatch_reported' in df.columns
    sample_size = df.loc[(df.dispatch_reported) & (df.event_type == eventtype)].shape[0] # is this right? or should it be df.shape[0]?
    obs_n = [df.loc[(df.numeric_district == district) & (df.event_type == eventtype),
             'dispatch_reported'].sum()/sample_size
             for district in districts]
    dof = len(districts) - 1 - ddof
    #nobs = sum(obs_n) # why use this and not sample_size?
    pcs = power_divergence(f_obs=obs_n, ddof=ddof, lambda_ = "pearson")
    chi_info = f"""This test will compare whether observed dispatch rates for reported {eventtype} events are equal across districts. In interpreting the results, a p-value below {sig} will be considered statistically significant. In such cases where the p-value is statistically significant, the effect size will be measured and categorized as small, medium, or large.\n"""
    suppinfo = text_statistic(pcs, sig=sig, n=sample_size, dof=dof,
                             null_phrase=f"the distribution by district of reported {eventtype} events with a reported dispatch date are equal")
    info = chi_info + suppinfo + "\n\n\n"
    return obs_n, pcs, info

In [4]:
# main
oemc = pd.read_parquet("../../data/OEMC_MP/export/output/oemc-prepped.parquet")
rules = readyaml("../../data/shared/hand/keywords.yml")
assert oemc.shape[0] > 12000000
assert not oemc.event_no.duplicated().any()
assert (oemc.dispatch_reported == 0).sum() == oemc.time_to_dispatch.isna().sum()

# preview data

In [4]:
rules['help']

{'gun': ['SHOTS FIRED', 'PERSON SHOT'],
 'mp': ['CHILD ABDUCTION', 'MISSING PER. TENDER AGE', 'MISSING PERSON'],
 'injury_report': ['PERSON DOWN',
  'INJURED PERSON REPORT',
  'CHILD ABUSE',
  'DOMESTIC BATTERY',
  'BATTERY',
  'ASSAULT',
  'EMS'],
 'noinjury_report': ['PERSON WITH A GUN',
  'PERSON CALLING FOR HELP',
  'ASSIST CITIZEN',
  'CHILD LEFT ALONE',
  'CHECK WELL BEING',
  'VIOLATION ORDER OF PROT',
  'MENTAL HEALTH DISTURBANCE',
  'DOMESTIC DISTURBANCE',
  'ALARM BURGLAR',
  'THEFT REPORT',
  'AUTO ACCIDENT PD',
  'AUTO ACCIDENT PI',
  'RECKLESS DRIVING']}

In [5]:
oemc.sample().T

,9940018
event_no,2110504845
district,019
call_date,2021-04-15 10:47:00
disp_date,2021-04-15 10:50:00
on_date,NaT
clear_date,2021-04-15 11:36:00
close_date,2021-04-15 11:36:00
init_priority,1A
init_type,CHECK WELL BEING
fin_type,CHECK WELL BEING


# light prep

In [6]:
# not sure if this is a useful alternative to what we already have in `ttd_group`
#oemc['ttd_group_short'] = oemc.time_to_dispatch.apply(regroup_td)

In [5]:
maincols = [
    'event_no', 'district', 'numeric_district',
    'call_date', 'year_called',
    'dispatch_reported', 'disp_date',
    'time_to_dispatch', 'ttd_group', #'ttd_group_short',
    'init_priority', 'init_type', 'fin_type',
    'event_group', 'event_type'
]
less = oemc[maincols].copy()
assert not less.event_no.duplicated().any()
#less['dispatch_oncall'] = less.call_date == less.disp_date
#less['dispatch_under5min'] = less.time_to_dispatch < pd.Timedelta(minutes=5)
#less['dispatch_under15min'] = less.time_to_dispatch < pd.Timedelta(minutes=15)
#less['dispatch_under30min'] = less.time_to_dispatch < pd.Timedelta(minutes=30)
#less['dispatch_under60min'] = less.time_to_dispatch < pd.Timedelta(minutes=60)
#less['dispatch_under2hr'] = less.time_to_dispatch < pd.Timedelta(minutes=120)
#less['dispatch_under24hr'] = less.time_to_dispatch < pd.Timedelta(days=1)
#less['dispatch_over24hr'] = less.time_to_dispatch >= pd.Timedelta(days=1)
#less['event_mp'] = less.event_type == 'mp'
#less['event_shotspotter'] = less.event_type == 'shotspotter'
#less['event_gun'] = less.event_type == 'gun'
#less['event_traffic'] = less.event_type == 'traffic'
#less['event_help'] = less.event_group == 'help'

# stage focus

In [6]:
districts = [
    2,
    7, # district 7 includes Englewood per WTTW map and has 5th highest event count
    8, # district 8 has the 2nd highest event count
    9,
    19,
    20,
    24
]
for num in districts:
    less[f'district_{num}'] = less.numeric_district == num
less['district_other'] = ~less.numeric_district.isin(districts)
distinds = [
    'district_2', 'district_7', 'district_8', 'district_9',
    'district_19', 'district_20', 'district_24', 'district_other']

In [7]:
less[distinds].sum()

district_2         601269
district_7         632357
district_8         686940
district_9         471412
district_19        455477
district_20        277942
district_24        360724
district_other    8673461
dtype: int64

# chi-square tests

In [8]:
overall_obs_n, overall_pcs, overall_info = get_chisquare_overall(df=oemc, districts=districts, sig=0.05, ddof=0)

In [28]:
overall_obs_n

[np.float64(0.0508177586387958),
 np.float64(0.05353844647590656),
 np.float64(0.05596415466328361),
 np.float64(0.039526643150483445),
 np.float64(0.03540925636782145),
 np.float64(0.02289682794290569),
 np.float64(0.02921837321872741)]

In [29]:
overall_pcs

Power_divergenceResult(statistic=np.float64(0.023809817415844422), pvalue=np.float64(0.9999997212913578))

In [30]:
print(overall_info)

This test will compare whether observed dispatch rates are equal across districts. In interpreting the results, a p-value below 0.05 will be considered statistically significant. In such cases where the p-value is statistically significant, the effect size will be measured and categorized as small, medium, or large.

This test results in a p-value of 1.0, which is not statistically significant and fails to reject the null hypothesis that the distribution by district of emergency events with a reported dispatch date are equal. 





In [31]:
shotspotter_obs_n, shotspotter_pcs, shotspotter_info = get_chisquare_event(df=oemc, districts=districts, eventtype='shotspotter', sig=0.05, ddof=0)

In [32]:
shotspotter_obs_n

[np.float64(0.03775262071643133),
 np.float64(0.08862414941659685),
 np.float64(0.09114780227639618),
 np.float64(0.08025624782884117),
 np.float64(5.1086090279338745e-05),
 np.float64(2.0434436111735498e-05),
 np.float64(3.065165416760325e-05)]

In [33]:
shotspotter_pcs

Power_divergenceResult(statistic=np.float64(0.2667663628379198), pvalue=np.float64(0.9996420253873786))

In [34]:
print(shotspotter_info)

This test will compare whether observed dispatch rates for reported shotspotter events are equal across districts. In interpreting the results, a p-value below 0.05 will be considered statistically significant. In such cases where the p-value is statistically significant, the effect size will be measured and categorized as small, medium, or large.

This test results in a p-value of 1.0, which is not statistically significant and fails to reject the null hypothesis that the distribution by district of reported shotspotter events with a reported dispatch date are equal. 





In [35]:
gun_obs_n, gun_pcs, gun_info = get_chisquare_event(df=oemc, districts=districts, eventtype='gun', sig=0.05, ddof=0)

In [36]:
gun_obs_n

[np.float64(0.039815496972997216),
 np.float64(0.06347630182474348),
 np.float64(0.05987806570784885),
 np.float64(0.059205398421901194),
 np.float64(0.01991308711575216),
 np.float64(0.013592150079545576),
 np.float64(0.03204245277982425)]

In [37]:
gun_pcs

Power_divergenceResult(statistic=np.float64(0.06005976282448802), pvalue=np.float64(0.9999955869830262))

In [38]:
print(gun_info)

This test will compare whether observed dispatch rates for reported gun events are equal across districts. In interpreting the results, a p-value below 0.05 will be considered statistically significant. In such cases where the p-value is statistically significant, the effect size will be measured and categorized as small, medium, or large.

This test results in a p-value of 1.0, which is not statistically significant and fails to reject the null hypothesis that the distribution by district of reported gun events with a reported dispatch date are equal. 





In [39]:
mp_obs_n, mp_pcs, mp_info = get_chisquare_event(df=oemc, districts=districts, eventtype='mp', sig=0.05, ddof=0)

In [40]:
mp_obs_n

[np.float64(0.04701279097481528),
 np.float64(0.06389528879002145),
 np.float64(0.047807261460236755),
 np.float64(0.03630730118376102),
 np.float64(0.03398347501390323),
 np.float64(0.07084690553745929),
 np.float64(0.09154286168268849)]

In [41]:
mp_pcs

Power_divergenceResult(statistic=np.float64(0.04589983885706685), pvalue=np.float64(0.9999980197443001))

In [42]:
print(mp_info)

This test will compare whether observed dispatch rates for reported mp events are equal across districts. In interpreting the results, a p-value below 0.05 will be considered statistically significant. In such cases where the p-value is statistically significant, the effect size will be measured and categorized as small, medium, or large.

This test results in a p-value of 1.0, which is not statistically significant and fails to reject the null hypothesis that the distribution by district of reported mp events with a reported dispatch date are equal. 





# checking distribution

In [44]:
oemc[['dispatch_reported', 'event_type']].value_counts()

dispatch_reported  event_type          
True               traffic                 3027734
                   other                   2354114
                   patrol                  2184342
                   noinjury_report         1778593
                   injury_report            778525
                   general                  668437
False              noinjury_report          442269
                   other                    369115
                   shotspotter_reclass?     141364
True               shotspotter               97874
                   gun                       93657
False              injury_report             71461
True               mp                        50348
                   shotspotter_reclass?      47999
                   hunchlab                  28731
False              general                    8884
                   mp                         7529
                   patrol                     4678
True               licplate               

In [50]:
oemc.dispatch_reported.value_counts(normalize=True)

dispatch_reported
True     0.913899
False    0.086101
Name: proportion, dtype: float64

In [54]:
(oemc[['district', 'dispatch_reported']].groupby('district').sum() / oemc[['district', 'dispatch_reported']].groupby('district').count()).mean()

dispatch_reported    0.734125
dtype: float64

In [53]:
oemc[['district', 'dispatch_reported']].groupby('district').sum() / oemc[['district', 'dispatch_reported']].groupby('district').count()

,dispatch_reported
district,
001,0.882418
002,0.939212
003,0.904822
004,0.941535
005,0.909355
006,0.914524
007,0.940850
008,0.905332
009,0.931765


In [46]:
oemc.loc[oemc.event_type == 'shotspotter', 'dispatch_reported']

190         True
224         True
241         True
255         True
259         True
            ... 
12159525    True
12159528    True
12159530    True
12159547    True
12159571    True
Name: dispatch_reported, Length: 97889, dtype: bool